In [1]:
import sys
import os
import pyam

import silicone.plotting as plotCor

<IPython.core.display.Javascript object>

In [2]:
# Inputs to the code for PlotCorrelationsBetweenGases, freely modifiable
# ________________________________________________________

years_of_interest = [2030]
save_results = '../Output/'
# if non-null, also plot these quantiles.
plot_quantiles = [0.2, 0.33, 0.5, 0.67, 0.8]
# if non-null, save data on the quantiles too
quantiles_savename = '../Output/regions/'
# How many boxes are used to fit the quantiles?
quantile_boxes = 15
# Should we extend the quantile boxes by an additional factor?
quantile_decay_factor = 0.7
# Color different models different colours?
model_colours = True
# In the model-coloured version, how much does the figure need to be reduced by to leave room for the legend?
legend_fraction = 0.65
# ________________________________________________________


In [16]:
def get_sr15_scenarios(output_file):
    conn = pyam.iiasa.Connection("iamc15")
    valid_model_ids = [
        "MESSAGE*",
        "AIM*",
        #"C-ROADS*",
        # "GCAM*", # These are commented out for brevity of runs
        # "IEA*",
        # "IMAGE*",
        # "MERGE*",
        # "POLES*",
        # "REMIND*",
        # "Reference*",  # missing required columns, not sure why
        # "Shell*",  # missing required columns, not sure why
        "WITCH*"
        # "GENeSYS-MOD",  # can't download, not sure why
    ]
    variables_to_fetch = [
        "Emissions*", "Price*"
    ]
    for model in valid_model_ids:
        print("Fetching data for {}".format(model))
        for variable in variables_to_fetch:
            print("Fetching {}".format(variable))
            var_df = conn.query(model=model, variable=variable)
            try:
                df.append(var_df, inplace=True)
            except NameError:
                df = pyam.IamDataFrame(var_df)

    print("Writing to {}".format(output_file))
    df.to_csv(output_file)
print('sr15 loading function now written')

sr15 loading function now written


In [17]:
# NBVAL_IGNORE_OUTPUT
SR15_SCENARIOS = './sr15_scenarios_more_regions.csv'
if not os.path.isfile(SR15_SCENARIOS):
    get_sr15_scenarios(SR15_SCENARIOS)
print('sr15 file available to load')

INFO:root:You are connected to the iamc15 scenario explorer. Please cite as:

D. Huppmann, E. Kriegler, V. Krey, K. Riahi, J. Rogelj, S.K. Rose, J. Weyant, et al., IAMC 1.5C Scenario Explorer and Data hosted by IIASA. IIASA & IAMC, 2018. doi: 10.22022/SR15/08-2018.15429, url: data.ene.iiasa.ac.at/iamc-1.5c-explorer


Fetching data for MESSAGE*
Fetching Emissions*
Fetching Price*
Fetching data for AIM*
Fetching Emissions*
Fetching Price*
Fetching data for WITCH*
Fetching Emissions*
Fetching Price*
Writing to ./sr15_scenarios_more_regions.csv
sr15 file available to load


In [18]:
sr15_data = pyam.IamDataFrame(SR15_SCENARIOS)
sr15_data.head()

INFO:root:Reading `./sr15_scenarios_more_regions.csv`


,model,scenario,region,variable,unit,year,value
0,AIM/CGE 2.0,ADVANCE_2020_1.5C-2100,R5ASIA,Emissions|BC,Mt BC/yr,2005,3.4665
1,AIM/CGE 2.0,ADVANCE_2020_1.5C-2100,R5ASIA,Emissions|BC,Mt BC/yr,2010,3.0156
2,AIM/CGE 2.0,ADVANCE_2020_1.5C-2100,R5ASIA,Emissions|BC,Mt BC/yr,2015,2.8850
3,AIM/CGE 2.0,ADVANCE_2020_1.5C-2100,R5ASIA,Emissions|BC,Mt BC/yr,2020,2.7671
4,AIM/CGE 2.0,ADVANCE_2020_1.5C-2100,R5ASIA,Emissions|BC,Mt BC/yr,2025,2.1161


In [25]:
sr15_data.filter(variable='Price*').variables()

0                                          Price|Carbon
1                             Price|Primary Energy|Coal
2                              Price|Primary Energy|Gas
3                              Price|Primary Energy|Oil
4                    Price|Secondary Energy|Electricity
5     Price|Agriculture|Non-Energy Crops and Livesto...
6              Price|Agriculture|Non-Energy Crops|Index
7                Price|Secondary Energy|Liquids|Biomass
8                     Price|Agriculture|Livestock|Index
9                          Price|Agriculture|Corn|Index
10                      Price|Agriculture|Soybean|Index
11                        Price|Agriculture|Wheat|Index
12           Price|Final Energy|Residential|Electricity
13     Price|Final Energy|Residential|Gases|Natural Gas
14       Price|Final Energy|Residential|Liquids|Biomass
15           Price|Final Energy|Residential|Liquids|Oil
16           Price|Final Energy|Residential|Solids|Coal
17                       Price|Secondary Energy|

In [29]:
sr15_data_asia = sr15_data.filter(region='R5ASIA')
sr15_data_asia.variables()

0                                          Emissions|BC
1                                         Emissions|CH4
2                                   Emissions|CH4|AFOLU
3                  Emissions|CH4|Energy|Demand|Industry
4     Emissions|CH4|Energy|Demand|Residential and Co...
5            Emissions|CH4|Energy|Demand|Transportation
6                           Emissions|CH4|Energy|Supply
7                                          Emissions|CO
8                                         Emissions|CO2
9                                   Emissions|CO2|AFOLU
10        Emissions|CO2|Energy and Industrial Processes
11                          Emissions|CO2|Energy|Demand
12                    Emissions|CO2|Energy|Demand|AFOFI
13                 Emissions|CO2|Energy|Demand|Industry
14    Emissions|CO2|Energy|Demand|Residential and Co...
15           Emissions|CO2|Energy|Demand|Transportation
16                          Emissions|CO2|Energy|Supply
17                                  Emissions|CO

In [33]:
# We don't seem to have a carbon price for regions
sr15_data_asia.filter(variable='Price*').variables()
sr15_data.filter(variable='Price|Carbon').filter(region='World', keep=False)

In [ ]:
plotCor._plot_emission_correlations_quantile_rolling_windows(sr15_data_asia, years_of_interest, save_results, 
                                plot_quantiles, quantiles_savename, quantile_boxes, quantile_decay_factor, model_colours,
                                legend_fraction)